# Customer Service Data Clustering

In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
import os
import re

import matplotlib.pyplot as plt
import matplotlib.cm as cm

from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from MulticoreTSNE import MulticoreTSNE as TSNE

import pickle

from tqdm import tqdm

from sklearn.cluster import DBSCAN

### Configuration

In [2]:
path = os.path.join("..","data","./newdata_clean.xlsx")
n_cpu = 15
batch_size = 10000
max_k = 40
max_features = 256  # only consider the top max_features ordered by term frequency across the corpus.
loadpath = "processed_data_lda_wo_html"
#loadpath = "processed_data_not_rmsw"

### Read Data

In [3]:
df = pd.read_excel(path)

df = df.dropna() # drop nan entry
# df[pd.isnull(df).any(axis=1)]

le = preprocessing.LabelEncoder()
le.fit(df['catName'].unique())
num_classes = len(le.classes_)
class_list = list(le.classes_)

#print(class_list)

print("number of classes:",num_classes)
df.loc[:,'catName'] = le.transform(df.loc[:,'catName'])
data = df.question

number of classes: 64


### Preprocess Data

In [4]:
def find_optimal_clusters(data, max_k, batch_size):
    iters = range(2, max_k+1, 2)
    
    sse = []
    bar = tqdm(iters)
    for k in bar:
        sse.append(MiniBatchKMeans(n_clusters=k, batch_size=batch_size, random_state=20).fit(data).inertia_)
        bar.set_description('Fit {} clusters'.format(k))
        
    f, ax = plt.subplots(1, 1)
    ax.plot(iters, sse, marker='o')
    ax.set_xlabel('Cluster Centers')
    ax.set_xticks(iters)
    ax.set_xticklabels(iters)
    ax.set_ylabel('SSE')
    ax.set_title('SSE by Cluster Center Plot')

In [5]:
def cluster_and_plot(data, cluster_algo, n_clusters, eps, calculate_portion, draw_portion):
    n_data = data.shape[0]
    print("Input Data shape:", data.shape)
    
    # random sample (n_sample) points
    np.random.seed(5)
    n_sample = n_data // calculate_portion
    print("Only calculate {} data points".format(n_sample))
    sample_items = np.random.choice(range(n_data), size=n_sample, replace=False)
    
    
    if cluster_algo == "kmeans":
        labels = MiniBatchKMeans(n_clusters=n_clusters, batch_size=batch_size, random_state=20).fit_predict(data)
        npdata = data[sample_items,:]
        label_subset = labels[sample_items]
    elif cluster_algo == "dbscan":
        npdata = data[sample_items]
        labels = DBSCAN(eps=eps, n_jobs=n_cpu).fit_predict(npdata)
        label_subset = labels
        print("Number of noise data: {}".format(len(label_subset[label_subset == -1])))
    
    # number of clusters
    max_label = max(labels) 
    print("Number of clusters: {}".format(max_label + 1))
    pca = PCA(n_components=2, whiten=True).fit_transform(npdata)
    print("PCA done")
    tsne_pca = TSNE(n_jobs=n_cpu, n_iter=5000).fit_transform(PCA(n_components=(max_features // 2)).fit_transform(npdata))
    print("tsne_pca done")
    tsne = TSNE(n_jobs=n_cpu, n_iter=5000).fit_transform(npdata)
    print("tsne30 done")
    tsne_per50 = TSNE(n_jobs=n_cpu, perplexity=50, n_iter=5000).fit_transform(npdata)
    print("tsne50 done")
    
    # draw only (n_draw) points
    np.random.seed(5)
    n_draw = n_sample // draw_portion
    print("Only draw {} data points".format(n_draw))
    idx = np.random.choice(range(pca.shape[0]), size=n_draw, replace=False)
    
    # draw scatter
    if cluster_algo == "kmeans":
        start_idx = 0
    else:
        start_idx = -1
    f, ax = plt.subplots(2, 2, figsize=(13, 10))
    for i in range(start_idx,max_label + 1):
        sub_idx = idx[label_subset[idx] == i]
        #print("{} points in group {}".format(len(sub_idx),i))
        label_subset_color = np.array([cm.hsv(i/ ( max_label + 1)) for i in label_subset[sub_idx]])
        ax[0,0].scatter(pca[sub_idx, 0], pca[sub_idx, 1])   # , c=label_subset_color
        ax[0,0].set_title('PCA Cluster Plot')
        
        ax[1,0].set_title('t-SNE & PCA Cluster Plot')
        ax[1,0].scatter(tsne_pca[sub_idx, 0], tsne_pca[sub_idx, 1], label="Group {} | {}".format(i,len(sub_idx))) # , c=label_subset_color
        
        ax[0,1].set_title('t-SNE Cluster Plot')
        ax[0,1].scatter(tsne[sub_idx, 0], tsne[sub_idx, 1]) # , c=label_subset_color
        
        ax[1,1].set_title('t-SNE Cluster Plot (Perplexity 50)')
        ax[1,1].scatter(tsne_per50[sub_idx, 0], tsne_per50[sub_idx, 1]) # , c=label_subset_color
    f.legend() # plot only one legend
    
    if cluster_algo == "kmeans":
        return data, labels
    else:
        return npdata, labels

In [87]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

import langid

from nltk.corpus import stopwords

namelist = ["paul", "jim", "larry", "ken", "wright", "peter", "donna", "ian", "rick", "richard", "william", "john", "chris", "tony", "joseph"]
stw = stopwords.words('english') + ["nbsp", "would", "cant", "attach", "file", "hey", "quot", "dont", "cyberlink", "guy", "wont", "didnt", "doesnt"]
#+ ['nbsp', 'powerdirector', 'cyberlink', 'powerdvd', 'power', 'director', 'ba']
print(stw)
print("Stopwords length: {}".format(len(stw)))

def _filter(x):
    cut = 0
    redundant_pos = re.search(r"dear\s+valued\s+customer,", x)
    if redundant_pos != None:
        print("cut dear valued customer,")
        x = x[:redundant_pos.start()]
        cut = 1
    redundant_pos = re.search(r"order\s+confirmation\s+order\s+number:", x)
    if redundant_pos != None:
        print("cut order confirmation order number:")
        x = x[:redundant_pos.start()]
        cut = 1
    redundant_pos = re.search(r"<li>comment:</li>", x)
    if redundant_pos != None:
        print("cut <li>comment:</li>")
        x = x[redundant_pos.end():]
        cut = 1
    redundant_pos = re.search(r"time\s+of\s+this\s+report:", x)
    if redundant_pos != None:
        print("cut time of this report:")
        x = x[:redundant_pos.start()]
        cut = 1
    redundant_pos = re.search(r"this\s+correspondence\s+is\s+from", x)
    if redundant_pos != None:
        print("cut this correspondence is from")
        x = x[:redundant_pos.start()]
        cut = 1
    redundant_pos = re.search(r"forwarded\s+message", x)
    if redundant_pos != None:
        print("cut forwarded message")
        x = x[:redundant_pos.start()]
        cut = 1
    redundant_pos = re.search(r"(best)?(kind)?\s+regards", x)
    if redundant_pos != None:
        print("cut best|kind regards")
        x = x[:redundant_pos.start()]
        cut = 1
    redundant_pos = re.search(r"media\s+source\s+error\s+report", x)
    if redundant_pos != None:
        print("cut Media Source Error Report")
        x = x[:redundant_pos.start()]
        cut = 1
        
    x = re.sub(r'<[^<]*?/?>', ' ', x)                          # remove all html tag
    x = re.sub(r"\w{5}(-\w{5}){5}", "", x)                         # remove key tokex, ex: sd2kk-33j7v-na4m2-m8n5s-sjaxq-bramm
    x = re.sub(r'https?:\/\/[^ ]*', ' ', x)                        # remove all url
    x = re.sub(r'\S*@\S*\s?', ' ', x)                              # remove all email address
    x = re.sub(r'\S*\.\S*\s?', ' ', x, flags=re.IGNORECASE)        # remove all filename
    x = re.sub(r'[^a-z A-Z]', ' ', x)                              # remove all non-english alphabat
    x = re.sub(r"power\s+dvd", "powerdvd", x, flags=re.IGNORECASE)
    x = re.sub(r"power\s+director", "powerdirector", x, flags=re.IGNORECASE)
    x = re.sub(r"audio\s+director", "audiodirector", x, flags=re.IGNORECASE)
    x = re.sub(r"color\s+director ", "colordirector ", x, flags=re.IGNORECASE)
    x = re.sub(r"action\s+director", "actiondirector", x, flags=re.IGNORECASE)
    x = re.sub(r"makeup\s+director ", "makeupdirector ", x, flags=re.IGNORECASE)
    x = re.sub(r"director\s+suite", "directorsuite", x, flags=re.IGNORECASE)
    x = re.sub(r"audio\s+director", "audiodirector", x, flags=re.IGNORECASE)
    x = re.sub(r"photo\s+director", "photodirector ", x, flags=re.IGNORECASE)
    x = re.sub(r"blue?[-\s]*rays?", "bluray", x, flags=re.IGNORECASE)
    x = re.sub(r"power\s*(2|(to))\s*go", "power2go", x, flags=re.IGNORECASE)
    x = re.sub(r"cyber\s+link", "cyberlink", x, flags=re.IGNORECASE)
    x = re.sub(r"pdr", "powerdirector", x, flags=re.IGNORECASE)
    x = re.sub(r"pdvd", "powerdvd", x, flags=re.IGNORECASE)
    x = re.sub(r"pls", "please", x, flags=re.IGNORECASE)
    x = re.sub(r"add[-\s]*ons?", "addon", x, flags=re.IGNORECASE)
    x = re.sub(r"media[-\s]*suite", "mediasuite", x, flags=re.IGNORECASE)
    
    '''
    if cut:
        print(x)
    '''
    return x
'''
def _correct_word(text1):
    pattern = re.compile(r"(.)\1{2,}")
    text2 = pattern.sub(r"\1\1", text1) # reduce lengthening
    #if text1 != text2:
    #    print(text1, text2)
    text3 = spell(text2).lower() # spell correction
    #if text2 != text3:
    #    print(text2, text3)
    return text3
'''
def _get_wordnet_pos(tag):
    if tag =='J':
        return wordnet.ADJ
    elif tag =='V':
        return wordnet.VERB
    elif tag =='N':
        return wordnet.NOUN
    elif tag =='R':
        return wordnet.ADV
    else:
        return None

def _lemmatization(tokens):
    tagged_sent = pos_tag(tokens) # [('The', 'DT'), ('striped', 'JJ'), ('for', 'IN'), ('best', 'JJS')]
    ret = []
    for tag in tagged_sent:
        wordnet_pos = _get_wordnet_pos(tag[1][0]) or wordnet.NOUN
        ret.append(wnl.lemmatize(tag[0], pos=wordnet_pos))
        #print(tag[0],tag[1][0],wordnet_pos,ret[-1])
    return ret

def _remove_stopword(tokens):
    ret = []
    for word in tokens:
        if word in namelist:
            print("skip name {}".format(word))
            continue
        if word not in stw and len(word) > 2:
            ret.append(word)
    return ret

def preprocess(sentence):
    sentence = _filter(sentence.lower())
    tokens = nltk.word_tokenize(sentence)

    #tokens = [self._correct_word(word) for word in tokens] # spell correction
    tokens = _lemmatization(tokens) # lemmatization
    tokens = _remove_stopword(tokens) # remove stopwords
    s = " ".join(tokens)

    return s, tokens

def process_batch(batch):   
    clean_batch = []
    reduced_batch = []
    token_batch = []
    for ori_s in tqdm(batch):
        s = ori_s.lower()
        ret = langid.classify(s)
        if ret[0] != "en" and ret[1] < -100: # remove language other than english
            #print(ret)
            #print(s)
            continue

        processed, tokens = preprocess(s)
        if len(tokens) <= 20: # remove too short sentence
            #print(s,processed)
            continue
        #print(processed)
        #print(tokens)
        reduced_batch.append(ori_s)
        clean_batch.append(processed)
        token_batch.append(tokens)
    return clean_batch, reduced_batch, token_batch

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package punkt to
[nltk_data]     /home/student/05/b05505004/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/student/05/b05505004/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
clean_data = []
reduced_data = []
token_data = []

n_workers = n_cpu
from multiprocessing import Pool
ret = [None] * n_workers
n_data = len(data)
print("Origin Data length:",n_data)
with Pool(processes=n_workers) as pool:
    for i in range(n_workers):
        batch_start = (n_data // n_workers) * i 
        if i == n_workers - 1:
            batch_end = n_data
        else:
            batch_end = (n_data // n_workers) * (i + 1)
        batch = data[batch_start:batch_end]
        ret[i] = pool.apply_async(process_batch, [batch])
    pool.close()
    pool.join()

for result in ret:
    clean_batch, reduced_batch, token_batch = result.get()
    clean_data += clean_batch
    reduced_data += reduced_batch
    token_data += token_batch
print("done")

Origin Data length: 106478


  0%|          | 0/7106 [00:00<?, ?it/s]

cut best|kind regards


  0%|          | 22/7098 [00:05<2:35:57,  1.32s/it]

cut best|kind regards


...

 59%|█████▉    | 4179/7098 [30:15<19:26,  2.50it/s]

cut best|kind regards


 59%|█████▉    | 4177/7098 [30:15<25:44,  1.89it/s]

cut best|kind regards


 60%|██████    | 4272/7098 [30:16<22:12,  2.12it/s]

In [ ]:
output = {
    "clean_data": clean_data,
    "reduced_data": reduced_data,
    "token_data": token_data
}
with open(loadpath, "wb") as f:
    pickle.dump(output, f)

In [ ]:
with open(loadpath, "rb") as f:
    output = pickle.load(f)
clean_data = output["clean_data"]
reduced_data = output["reduced_data"]
token_data = output["token_data"]

## TF-IDF Clustering

In [ ]:
print("max_feature",max_features)
tfidf = TfidfVectorizer(
    min_df = 0.001,
    max_df = 0.95,
    max_features = max_features,
    stop_words = 'english'
)
tfidf.fit(clean_data)
text = tfidf.transform(clean_data)

In [ ]:
find_optimal_clusters(text, max_k, batch_size)

### Clustering by Kmeans

In [ ]:
sample_text, sample_text_labels = cluster_and_plot(text, "kmeans", 10, None, 500, 10)

### Clustering by DBSCAN

In [ ]:
cluster_and_plot(text, "dbscan", None, 0.999, 50, 10)

### Get keywords of each TF-IDF cluster
作法是將每個 cluster 的前幾高 tfidf 字取出來，作為這個 cluster 的代表字。但是效果不好，且很多 cluster 都有相同常見的字(ex: download)

In [ ]:
def get_top_keywords(data, text, clusters, labels, n_terms):
    data = np.array(data)
    clusters = np.array(clusters)
    text_feature = text.toarray()
    labels = np.array(labels)
    group = pd.DataFrame(text.todense()).groupby(clusters)
    #print("Data point in each group:\n", group.size())
    df = group.mean() # [(clusters) rows x (feature) columns]

    for i,r in df.iterrows():
        sub_text = data[clusters == i]
        sub_text_list = sub_text
        sub_text_feature = text_feature[clusters == i]
        #print("sub_text: {} | sub_text_feature: {}".format(sub_text.shape,sub_text_feature.shape))

        dist = np.linalg.norm(sub_text_feature - np.array(r), axis=1)

        #print("Euclidean distance:", dist.shape, dist)
        
        print('\nCluster {}'.format(i))
        print(', '.join([labels[t] for t in np.argsort(r)[-n_terms:]]))
        #print(','.join([str(r[t]) for t in np.argsort(r)[-n_terms:]]))
        print('\n'.join([re.sub(' +', ' ',_filter(sub_text_list[t])) + " | " + str(dist[t]) for t in np.argsort(dist)[-2:]]))
            
get_top_keywords(reduced_data, sample_text, sample_text_labels, tfidf.get_feature_names(), 10)

## Doc2vec Clustering

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
tagged_data = [TaggedDocument(words=tokens, tags=[str(i)]) for i, tokens in enumerate(token_data)]

max_epochs = 100
alpha = 0.025

model = Doc2Vec(vector_size=max_features,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                workers=n_cpu,
                dm=1)
  
model.build_vocab(tagged_data)
bar = tqdm(range(max_epochs))
for epoch in bar:
    bar.set_description('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

In [ ]:
#model= Doc2Vec.load("d2v.model")
model= Doc2Vec.load("enwiki_dbow/doc2vec.bin")
'''
#to find the vector of a document which is not in training data
test_data = word_tokenize("I love chatbots".lower())
v1 = model.infer_vector(test_data)
print("V1_infer", v1)


# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('1')
print(similar_doc)


# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
print(model.docvecs['1'])
'''

n_reduced_data = len(reduced_data)
docvec = []
for i in tqdm(range(n_reduced_data)):
    #docvec.append(model.docvecs[str(i)])
    docvec.append(model.infer_vector(token_data[i]))
docvec = np.array(docvec)
print("Number of data: {}".format(n_reduced_data))

In [ ]:
find_optimal_clusters(docvec, max_k, batch_size)

### Clustering by Kmeans

In [ ]:
_ = cluster_and_plot(docvec, "kmeans", 10, None, 50, 10)

### Clustering by DBSCAN

In [ ]:
_ = cluster_and_plot(docvec, "dbscan", None, 2.1, 50, 10)

### Reference
* [Clustering documents with TFIDF and KMeans](https://www.kaggle.com/jbencina/clustering-documents-with-tfidf-and-kmeans)
* [Analyzing tf-idf results in scikit-learn](https://buhrmann.github.io/tfidf-analysis.html)

## LDA Topic Model
如果不移除 stopword 的話效果很差，主題的字都會是 of, for, it...
`dictionary.filter_extremes()` 過濾掉 token 出現次數少於15個句子，或是出現在超過一半的句子中。 
ref: [Topic Modeling and Latent Dirichlet Allocation (LDA) in Python](https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24)

In [35]:
import gensim
from gensim.models import LdaMulticore
import pprint
print("Data length: {}".format(len(token_data)))
dictionary = gensim.corpora.Dictionary(token_data)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

bow_corpus = [dictionary.doc2bow(doc) for doc in token_data]
print(bow_corpus[4310]) # just print

Data length: 37493
[(2, 2), (12, 1), (16, 1), (17, 1), (25, 1), (32, 1), (34, 1), (39, 1), (45, 1), (48, 1), (49, 1), (56, 1), (62, 1), (78, 1), (149, 1), (172, 1), (177, 1), (206, 1), (245, 1), (438, 1), (563, 1)]


### LDA using BOW

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=n_cpu)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

### LDA using TF-IDF

In [86]:
from gensim import corpora, models
from gensim.models.coherencemodel import CoherenceModel

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
print(corpus_tfidf[4310])

model_list = []
perplexity = []
coherence = []

n_topic = 80
test = [n_topic]
for i in test:
    print(i)
    lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=i, iterations=100000, id2word=dictionary, passes=2, workers=n_cpu)
    model_list.append(lda_model_tfidf)
    
    for idx, topic in lda_model_tfidf.print_topics(-1):
        print('Topic: {} \nWord: {}'.format(idx, topic))
    print("=========================================================================================")
    
    # Compute Perplexity
    per_score = lda_model_tfidf.log_perplexity(corpus_tfidf)
    print('Perplexity: ', per_score)  # a measure of how good the model is. lower the better.
    perplexity.append(per_score)

    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model_tfidf, texts=token_data, dictionary=dictionary, coherence='c_v', processes=n_cpu)
    coherence_lda = coherence_model_lda.get_coherence() # high is better
    print('Coherence Score: ', coherence_lda)
    coherence.append(coherence_lda)
'''
with open("perplexity", "wb") as f:
    pickle.dump(perplexity, f)
with open("coherence", "wb") as f:
    pickle.dump(coherence, f)
with open("model_list", "wb") as f:
    pickle.dump(model_list, f)


f, ax = plt.subplots(1, 2, figsize=(10, 5))
x = range(1,500)
ax[0].plot(x,perplexity)   # , c=label_subset_color
ax[0].set_title('BOW Topic Distribution')
ax[1].plot(coherence)   # , c=label_subset_color
ax[1].set_title('coherence')
'''

[(2, 0.2531097459006541), (12, 0.21288620502306962), (16, 0.23524732383283709), (17, 0.1973247914660445), (25, 0.28786039145768494), (32, 0.22733785834350081), (34, 0.11603808641497698), (39, 0.2000474632219321), (45, 0.1811068019251443), (48, 0.19556975293695888), (49, 0.3054919151956636), (56, 0.11998465513873147), (62, 0.16259132685567054), (78, 0.17334701841435926), (149, 0.29491224994819093), (172, 0.2736302516600575), (177, 0.131740063961617), (206, 0.18216470860396947), (245, 0.1191492070596504), (438, 0.297794323479943), (563, 0.24039235782872304)]
80
Topic: 0 
Word: 0.014*"morning" + 0.012*"powerdirect" + 0.010*"tracker" + 0.010*"property" + 0.008*"shortcut" + 0.008*"motion" + 0.008*"burn" + 0.008*"alan" + 0.007*"label" + 0.006*"computer"
Topic: 1 
Word: 0.010*"video" + 0.010*"fps" + 0.007*"frame" + 0.007*"retire" + 0.007*"native" + 0.006*"iphone" + 0.006*"play" + 0.006*"center" + 0.006*"use" + 0.005*"clip"
Topic: 2 
Word: 0.069*"membership" + 0.024*"programme" + 0.017*"shot" 

Perplexity:  -10.955453269870732
Coherence Score:  nan


'\nwith open("perplexity", "wb") as f:\n    pickle.dump(perplexity, f)\nwith open("coherence", "wb") as f:\n    pickle.dump(coherence, f)\nwith open("model_list", "wb") as f:\n    pickle.dump(model_list, f)\n\n\nf, ax = plt.subplots(1, 2, figsize=(10, 5))\nx = range(1,500)\nax[0].plot(x,perplexity)   # , c=label_subset_color\nax[0].set_title(\'BOW Topic Distribution\')\nax[1].plot(coherence)   # , c=label_subset_color\nax[1].set_title(\'coherence\')\n'

### Visualize

In [82]:
def print_topic_example(reduced_data, token_data, model, dictionary, n_topic):
    total_len = len(reduced_data)
    print("total_len",total_len)

    topic_distribution = [0 for i in range(n_topic)]
    topic_result = []
    example = [[] for i in range(n_topic)]
    for s, token in tqdm(zip(reduced_data, token_data), total=total_len):
        bow_vector = dictionary.doc2bow(token)
        rank = model[bow_vector]
        if len(rank) == 0:
            continue
        index, score = max(rank, key=lambda tup: tup[1])
        #print(index,score)
        topic_distribution[index] += 1
        topic_result.append(index)
        example[index].append((s, score))
        #print(s)
        #print("Score: {}  Topic: {}\n".format(score, model.print_topic(index, 5)))
    print("topic_distribution: {}".format(topic_distribution))
    
    for idx, topic in model.print_topics(-1):
        print('Topic: {} | {} datas\nWord: {}'.format(idx, topic_distribution[idx], topic))
        result = sorted(example[idx], key=lambda tup: -tup[1])[:5]
        for s, score in result:
            print("{} | {}\n".format(s,score))
        print()
        print("====================")
    
    return topic_result

In [ ]:
bow_topic_result = print_topic_example(reduced_data, token_data, lda_model, dictionary, n_topic)

In [83]:
lda_model_tfidf = model_list[0]
print(lda_model_tfidf)
tfidf_topic_result = print_topic_example(reduced_data, token_data, lda_model_tfidf, dictionary, n_topic)

  0%|          | 71/37080 [00:00<01:23, 443.91it/s]

LdaModel(num_terms=3502, num_topics=110, decay=0.5, chunksize=2000)
total_len 37080


100%|██████████| 37080/37080 [00:55<00:00, 668.63it/s] 


topic_distribution: [3, 1, 1, 8, 0, 386, 6186, 3, 42, 1, 3, 0, 0, 13490, 1, 0, 2, 33, 16, 1, 2, 2, 4, 0, 2619, 4, 752, 59, 15, 5, 126, 29, 0, 161, 216, 19, 3, 0, 0, 34, 11, 58, 2, 0, 3, 0, 3994, 0, 21, 0, 4, 1796, 6, 81, 1, 0, 1, 9, 3, 569, 4, 0, 15, 86, 7, 267, 0, 152, 0, 861, 0, 5, 1, 1, 0, 4, 1388, 61, 0, 26, 627, 10, 1, 1, 0, 4, 27, 1, 1, 6, 18, 1, 1891, 10, 3, 1, 0, 294, 0, 0, 10, 2, 3, 14, 303, 0, 179, 1, 7, 0]
Topic: 0 | 3 datas
Word: 0.363*"burn" + 0.090*"recording" + 0.062*"record" + 0.030*"protect" + 0.028*"reformat" + 0.026*"jpeg" + 0.022*"movie" + 0.022*"commercial" + 0.021*"editor" + 0.017*"unsuccessful"
I am recording home movie from VHS.    My recordings are stopped prematurely with the error code: Copy Right - Recording Prohibited    Therefore I cannot capture these recordings    Is there anyway to let the software know that I am recording home movies and not violating any copyright laws? | 0.3139496445655823

When burning a MP3 CD it won't burn the songs in the order I

In [65]:
import pyLDAvis.gensim

visual = pyLDAvis.gensim.prepare(lda_model_tfidf, corpus_tfidf, dictionary)
pyLDAvis.display(visual)

/tmp2/b05505004/intern/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [ ]:
f, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].hist(bow_topic_result, rwidth=0.8)   # , c=label_subset_color
ax[0].set_title('BOW Topic Distribution')
ax[1].hist(tfidf_topic_result, rwidth=0.8)   # , c=label_subset_color
ax[1].set_title('TFIDF Topic Distribution')

## Bert Sentence Encoding Clustering

1. 要先把 bert server 開起來
    `gpu0 bert-serving-start -model_dir uncased_L-12_H-768_A-12 -num_worker 4  -port 1355 -max_seq_len 40 -device_map 0`
2. 直到出現 `all set, ready to serve request!` 才可以 run client 的 command
Note:
1. 如果是 run 大的 model 會 load 不進去 GPU 中，跑出 OOM
2. 如果系統記憶體不夠則不會出現 all set, ready to serve request!，CPU 版來說，最多只能 -num_worker 2

In [ ]:
from bert_serving.client import BertClient
bc = BertClient(port=1355)
print("Start predicting")
bert_output = bc.encode(clean_data)
bert_output[:10]

In [ ]:
bert_data_path = "bert_base.pkl"

In [ ]:
bert_data = {
    "clean_data": clean_data,
    "reduced_data": reduced_data,
    "token_data": token_data,
    "bert_data": bert_output
}
with open(bert_data_path, "wb") as f:
    pickle.dump(bert_data, f)

In [ ]:
with open(bert_data_path, "rb") as f:
    bert_data = pickle.load(f)
clean_data = bert_data["clean_data"]
reduced_data = bert_data["reduced_data"]
token_data = bert_data["token_data"]
bert_output = bert_data["bert_data"]

In [ ]:
find_optimal_clusters(bert_output, max_k, batch_size)

### Clustering by Kmeans

In [ ]:
_ = cluster_and_plot(bert_output, "kmeans", 10, None, 50, 10)

### Clustering by DBSCAN

In [ ]:
_ = cluster_and_plot(bert_output, "dbscan", None, 5.95, 50, 10)